In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("train.csv")
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df.dropna(inplace=True)

In [8]:
df.reset_index(inplace=True)

In [9]:
df.index

RangeIndex(start=0, stop=18285, step=1)

In [10]:
import tensorflow as tf

In [13]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /home/dhakad/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
## Get the Independent Features

X=df.drop('label',axis=1)

In [15]:
## Get the Dependent features
y=df['label']

In [16]:
X.shape, y.shape

((18285, 5), (18285,))

In [17]:
messages=X.copy()

In [18]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [19]:
import re
from nltk.corpus import stopwords

In [21]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [24]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

### one_hot representation

In [26]:
voc_size = 5000

onehot_repr=[one_hot(words,voc_size)for words in corpus] 
onehot_repr

[[3277, 2859, 2510, 1509, 3080, 4419, 1565, 2509, 1285, 2310],
 [4337, 3, 3650, 1173, 2092, 4763, 813],
 [1339, 988, 2208, 2432],
 [4630, 3558, 2138, 4952, 970, 3868],
 [4510, 2092, 4488, 2528, 3658, 571, 2092, 4497, 2284, 1413],
 [71,
  660,
  1625,
  4436,
  3292,
  2693,
  2268,
  4515,
  891,
  2369,
  3428,
  2241,
  2914,
  2986,
  813],
 [484, 4638, 4064, 684, 4144, 2570, 4305, 2884, 13, 1417, 4784],
 [1165, 857, 2008, 2952, 212, 1252, 2693, 788, 13, 1417, 4784],
 [3962, 1490, 3627, 405, 4718, 911, 1300, 1524, 2693, 4665],
 [3067, 3427, 2327, 734, 4424, 4367, 2453, 2238],
 [2721, 4782, 865, 3582, 1451, 879, 492, 2182, 2689, 3929, 4058],
 [4952, 2695, 3080, 911, 2693, 212],
 [2981, 2989, 3116, 3598, 795, 2275, 297, 4475, 3822],
 [407, 1402, 67, 3445, 3175, 3728, 2340, 13, 1417, 4784],
 [1162, 3920, 274, 3097, 1350, 13, 1417, 4784],
 [21, 128, 4591, 393, 165, 745, 2607, 2477, 824, 1689],
 [1328, 2561, 3],
 [3963, 1636, 4093, 2224, 2693, 3052, 4653, 813],
 [2610, 4280, 3650, 2573, 

### Embedding representation

In [27]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 2509 1285 2310]
 [   0    0    0 ... 2092 4763  813]
 [   0    0    0 ...  988 2208 2432]
 ...
 [   0    0    0 ...   13 1417 4784]
 [   0    0    0 ... 2212 1438 2223]
 [   0    0    0 ... 4695 1138  108]]


In [28]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 3277,
       2859, 2510, 1509, 3080, 4419, 1565, 2509, 1285, 2310], dtype=int32)

In [29]:
## Creating model

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [30]:
len(embedded_docs),y.shape

(18285, (18285,))

In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [32]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

### Model Training

In [34]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 9s 27ms/step - loss: 0.5015 - accuracy: 0.7285 - val_loss: 0.2124 - val_accuracy: 0.9114
Epoch 2/10
192/192 [==============================] - 4s 21ms/step - loss: 0.1480 - accuracy: 0.9453 - val_loss: 0.2010 - val_accuracy: 0.9185
Epoch 3/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0932 - accuracy: 0.9648 - val_loss: 0.2238 - val_accuracy: 0.9085
Epoch 4/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0702 - accuracy: 0.9763 - val_loss: 0.2287 - val_accuracy: 0.9181
Epoch 5/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0406 - accuracy: 0.9874 - val_loss: 0.2990 - val_accuracy: 0.9191
Epoch 6/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0271 - accuracy: 0.9921 - val_loss: 0.3676 - val_accuracy: 0.9175
Epoch 7/10
192/192 [==============================] - 4s 21ms/step - loss: 0.0153 - accuracy: 0.9961 - val_loss: 0.4992 - val_accuracy: 0.9130

In [35]:
# performance metrics

y_pred=model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

/home/dhakad/anaconda3/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


array([[3169,  250],
       [ 311, 2305]])

In [37]:
# accuracy

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9070422535211268

### To Remove Overfitting, Add Dropout

In [38]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [39]:
### Finally Training
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 26ms/step - loss: 0.4861 - accuracy: 0.7450 - val_loss: 0.2129 - val_accuracy: 0.9097
Epoch 2/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1558 - accuracy: 0.9402 - val_loss: 0.1950 - val_accuracy: 0.9200
Epoch 3/10
192/192 [==============================] - 4s 22ms/step - loss: 0.1147 - accuracy: 0.9596 - val_loss: 0.2146 - val_accuracy: 0.9147
Epoch 4/10
192/192 [==============================] - 4s 22ms/step - loss: 0.0844 - accuracy: 0.9693 - val_loss: 0.2314 - val_accuracy: 0.9160
Epoch 5/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0681 - accuracy: 0.9760 - val_loss: 0.2536 - val_accuracy: 0.9162
Epoch 6/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0469 - accuracy: 0.9847 - val_loss: 0.2514 - val_accuracy: 0.9084
Epoch 7/10
192/192 [==============================] - 4s 23ms/step - loss: 0.0499 - accuracy: 0.9835 - val_loss: 0.3606 - val_accuracy: 0.8981

In [40]:
# performance metrics

y_pred=model.predict_classes(X_test)

from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[3112,  307],
       [ 252, 2364]])

In [41]:
# accuracy

from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9073736536868269